In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import graphviz

In [2]:
df = pd.read_csv('../master_data.csv')
df.head(3)

C:\Users\moyersjm\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3,4,5,12,13,14,15,17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,video_id,title,channel_id,channel_title,published_at,view_count,likes,dislikes,comment_count,...,description,comments,publishedAt,channelId,channelTitle,categoryId,trending_date,thumbnail_link,comments_disabled,ratings_disabled
0,0,--0bCF-iK2E,Jadon Sancho Magical Skills & Goals,UC6UL29enLNe4mqwTfAyeNuw,Bundesliga,2021-07-01 10:00:00,1048888,19515,226,1319,...,Enjoy the best skills and goals from Jadon San...,"Respect to Dortmund fans,must be sad losing hi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,--14w5SOEUs,Migos - Avalanche (Official Video),UCGIelM2Dj3zza3xyV3pL3WQ,MigosVEVO,2021-06-10 16:00:00,15352638,359277,7479,18729,...,"Watch the the official video for Migos - ""Aval...",Migos just makes me want to live my live to th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,--40TEbZ9Is,Supporting Actress in a Comedy: 73rd Emmys,UClBKH8yZRcM4AsRjDVEdjMg,Television Academy,2021-09-20 01:03:32,925281,11212,401,831,...,Hannah Waddingham wins the Emmy for Supporting...,Hannah's energy bursts through any screen. Wel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.channel_id.fillna(df.channelId,inplace=True)
df.channel_title.fillna(df.channelTitle,inplace=True)
df.published_at.fillna(df.publishedAt,inplace=True)

In [4]:
df.shape

(1032705, 21)

In [5]:
features = df.drop(['Unnamed: 0','video_id',
                    #'title',
                    'channel_id',
                    #'tags',
                    #'description',
                    #'comments',
                    'publishedAt',
                    'channelId', 'channelTitle',
                    'categoryId','trending_date',
                   'thumbnail_link','comments_disabled',
                   'ratings_disabled'], axis=1)
features = features.dropna()
#target = features.dislikes
#features = features.drop('dislikes',axis=1)
features.shape

(37264, 10)

In [6]:
features.head(3)

,title,channel_title,published_at,view_count,likes,dislikes,comment_count,tags,description,comments
0,Jadon Sancho Magical Skills & Goals,Bundesliga,2021-07-01 10:00:00,1048888,19515,226,1319,football soccer ftbol alemn Bundesliga season ...,Enjoy the best skills and goals from Jadon San...,"Respect to Dortmund fans,must be sad losing hi..."
1,Migos - Avalanche (Official Video),MigosVEVO,2021-06-10 16:00:00,15352638,359277,7479,18729,Migos Avalanche Quality Control Music/Motown R...,"Watch the the official video for Migos - ""Aval...",Migos just makes me want to live my live to th...
2,Supporting Actress in a Comedy: 73rd Emmys,Television Academy,2021-09-20 01:03:32,925281,11212,401,831,,Hannah Waddingham wins the Emmy for Supporting...,Hannah's energy bursts through any screen. Wel...


In [7]:
corpus = pd.concat([features.title, features.tags, features.description])
corpus.head(3)

0          Jadon Sancho  Magical Skills & Goals
1            Migos - Avalanche (Official Video)
2    Supporting Actress in a Comedy: 73rd Emmys
dtype: object

In [8]:
vectorizer = TfidfVectorizer(analyzer='word',stop_words='english')
corpus_features = vectorizer.fit_transform(corpus)

In [26]:
svd = TruncatedSVD(n_components=2000)
features_svd = svd.fit_transform(corpus_features)
print('unexplained variance: ',(1-svd.explained_variance_ratio_.sum()).round(4))

unexplained variance:  0.5169


In [27]:
features_svd.shape

(111792, 2000)

In [28]:
corpus.shape

(111792, 2)

In [34]:
df_features_svd = pd.DataFrame(features_svd)
corpus = pd.DataFrame(corpus)
corpus = corpus.reset_index()
corpus['video'] = corpus[0]
corpus = corpus.drop([0,'index'],axis=1)
df_svd = pd.concat([corpus,df_features_svd],axis=1)
df_svd.head(3)

,video,0,1,2,3,4,5,6,7,8,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,Jadon Sancho Magical Skills & Goals,0.003705,0.014242,0.037304,-0.010373,0.006422,0.000502,0.002867,0.011206,0.003821,...,-0.002681,0.006738,0.010829,0.006452,-0.011062,-0.001684,-0.002810,0.002730,0.004779,0.002892
1,Migos - Avalanche (Official Video),0.021080,-0.005811,0.008128,0.014892,0.027148,0.199894,-0.111995,0.070640,-0.116385,...,0.015899,0.012841,-0.015499,0.019741,0.004399,-0.005332,-0.005028,-0.011408,0.012427,0.004038
2,Supporting Actress in a Comedy: 73rd Emmys,0.002277,0.000217,0.002391,0.003033,0.001708,0.012825,0.002186,-0.000814,-0.000389,...,-0.002947,-0.004532,-0.008685,-0.000315,-0.003074,0.001064,-0.000337,-0.003914,0.001284,0.000966


In [36]:
k = 1
ix = [0,1,2,3,4,5,-5,-4,-3,-2,-1]
df_svd.sort_values([k],ascending=False).iloc[ix,:][['video',k]].round(2)

,video,1
78979,SUBSCRIBE http://bit.ly/SSFootballSub\nPREMIE...,0.86
97681,SUBSCRIBE http://bit.ly/SSFootballSub\nPREMIE...,0.86
74575,SUBSCRIBE http://bit.ly/SSFootballSub\nPREMIE...,0.85
79611,SUBSCRIBE http://bit.ly/SSFootballSub\nPREMIE...,0.85
80418,SUBSCRIBE http://bit.ly/SSFootballSub\nPREMIE...,0.85
111392,SUBSCRIBE http://bit.ly/SSFootballSub\nPREMIE...,0.85
92589,! 2 / ?! \n\n : https://www.adid...,-0.22
110542,\nhttps://youtu.be/eFdnIf1s3ZI\n\n\nhttps://ww...,-0.22
109509,\nhttps://www.youtube.com/watch?v=Gqfg0WTRHK8\...,-0.23
109918,"When something breaks or needs fixing, we all ...",-0.23


In [37]:
df_svd.iloc[78979]['video']

"SUBSCRIBE  http://bit.ly/SSFootballSub\nPREMIER LEAGUE HIGHLIGHTS  http://bit.ly/SkySportsPLHighlights2021\nHighlights from the Premier League, as Manchester United delayed Manchester City's Premier League coronation with a 2-0 win over their rivals. \n\nWatch Premier League LIVE on Sky Sports here  http://bit.ly/WatchSkyPL\nTWITTER: https://twitter.com/skysportsfootball\nFACEBOOK: http://www.facebook.com/skysports\nWEBSITE: http://www.skysports.com/football\n\nMORE FROM SKY SPORTS ON YOUTUBE:\nSKY SPORTS FOOTBALL: http://bit.ly/SSFootballSub\nSKY SPORTS BOXING: http://bit.ly/SSBoxingSub\nSKY SPORTS CRICKET: http://bit.ly/SubscribeSkyCricket\nSOCCER AM: http://bit.ly/SoccerAMSub\nSKY SPORTS F1: http://bit.ly/SubscribeSkyF1\nSKY SPORTS: http://bit.ly/SkySportsSub\nSKY SPORTS GOLF: https://bit.ly/SubscribeSkySportsGolf"

In [38]:
df_svd.iloc[81191]['video']

'Thank you.\n\nSupport me: https://www.patreon.com/user?u=9866256\nTwitch: https://www.twitch.tv/jocat\nMerch: https://www.redbubble.com/people/JoCat105/explore?page=1&sortOrder=recent \nHit Point Press: https://hitpointpress.com?aff=4\nTwitter: https://twitter.com/JoCat105\nStream VODs: https://www.youtube.com/channel/UCqfq3qAqk-VUaX5ZGm8Zbhw\nWebsite: https://www.jocat.net/\nDiscord: https://discord.gg/xFMsw2r\n\nSpecial Guests:\n-  | https://twitter.com/Takahata101 | https://www.youtube.com/user/takahata101\n-   | https://twitter.com/DistortionDevil | https://www.youtube.com/user/Sypher782\n-   | https://twitter.com/GaijinGoombah | https://www.youtube.com/user/GaijinGoomba\n-   | https://twitter.com/CzBacklash | https://www.youtube.com/user/CzBacklash\n-  | https://twitter.com/Octopimp | https://www.youtube.com/user/MasterYorgi\n-   | https://twitter.com/MontyGlu | https://www.youtube.com/user/MontyGlu\n-  | https://twitter.com/YaroShienVA | https://www.youtube.com/user/DragonMaster

So that's hilarious. The two polar opposite channels are Sky Sports Football and JoCat. Let's see what other insightes we can find

In [50]:
k = 70
ix = [0,1,2,3,4,5,-5,-4,-3,-2,-1]
df_svd.sort_values([k],ascending=False).iloc[ix,:][['video',k]].round(2)

,video,70
72636,jake paul,0.34
60961,jake paul,0.34
61639,jake paul,0.34
50600,jake paul,0.34
53321,jake paul,0.34
49078,jake paul,0.34
45719,KSI ksi number 2 ksi new music ksiolajidebthd ...,-0.27
36248,KSI - Holiday (The KSI Show),-0.27
66248,MusicVideo Lyrics KSI new music KSI Patience Y...,-0.27
56583,ksi ksi farm animals ksi craig david ksi reall...,-0.27


So apparently there's a big difference between Jake Paul and KSI. Huh. Their main connection is a boxing match between Logan Paul and KSI.

In [88]:
k = 131
df_svd.sort_values([k],ascending=False).iloc[ix,:][['video',k]].round(2)

,video,131
2567,It's Time To Open Up,0.26
23708,OUR FIRST TIME...,0.21
10156,Time to move!,0.21
101029,Emma Raducanu takes on Leylah Fernandez in the...,0.18
76509,Emma Raducanu takes on Shelby Rogers in Round ...,0.18
101902,An almost speechless Emma Raducanu gives her p...,0.18
11978,Baby Name Reveal!,-0.18
17414,OUR BABY'S NAME REVEAL! HIS NAME IS...,-0.18
31961,BABY NAME REVEAL,-0.18
52168,Little mix little mix not a pop song little mi...,-0.18


In [89]:
df_svd.iloc[23708]['video']

'OUR FIRST TIME...'

In [72]:
vocab = pd.Series(vectorizer.get_feature_names())

In [79]:
components = svd.components_
k = 124
components = pd.Series(components[k,:],index=vocab)
components.sort_values(ascending=False)[ix].round(2)

roblox     0.37
kids       0.16
stray      0.15
makeup     0.13
world      0.13
la         0.11
mv        -0.10
try       -0.12
warzone   -0.13
west      -0.14
cup       -0.19
dtype: float64